In [1]:
import datetime
import tarfile

import boto3
import pandas as pd
import numpy as np
from sagemaker import get_execution_role
import sagemaker
!pip install pmdarima


sm_boto3 = boto3.client('sagemaker')

sess = sagemaker.Session()

region = sess.boto_session.region_name

bucket = sess.default_bucket()  # this could also be a hard-coded bucket name

print('Using bucket ' + bucket)

Using bucket sagemaker-us-east-1-754307369999


In [18]:
import pandas as pd
from sagemaker.sklearn.estimator import SKLearn
import time
import boto3
import numpy as np
import pandas as pd
import sagemaker
from sagemaker.session import Session
!pip install xlrd
!pip install s3fs


sess = boto3.Session()
sm = sess.client('sagemaker')
role = sagemaker.get_execution_role()
import psycopg2 as pg
import pandas.io.sql as psql
import pandas as pd
connection = pg.connect(user = 'postgres',
password = 'sa2dbroot',
host = 'sa2-db.cvl1on5n4pwi.us-east-1.rds.amazonaws.com',
port = '5432',
database = 'yash_sa_schema')
#dataframe = psql.DataFrame("SELECT * FROM category", connection)
data = pd.read_sql_query('select * from yash_sa_schema.sa_file_ingst_log',con=connection)
print(data['published_file_path'].iloc[0])
#print(data['data_src_nm'].iloc[0])
print(data['published_file_name'].iloc[0])
print(data['Model'].iloc[0])
Type=data['file_upld_typ'].iloc[0]
model=data['Model'].iloc[0]
DatasetPath=data['published_file_path'].iloc[0]
filename=data['published_file_name'].iloc[0]
if model=='Arema':
    !aws s3 cp s3://sagemaker-us-east-1-754307369999/Models/Arema.py ./DataFolder/Arema.py
elif model=='lr':
    !aws s3 cp s3://sagemaker-us-east-1-754307369999/Models/lr.py ./DataFolder/lr.py
elif model=='Customer_Segmentation':
    import pandas as pd
    from datetime import timedelta
    from io import StringIO

    def process_execution_function(data, id_col, date_col, order_id_col, value_col):
        data[date_col] = pd.to_datetime(data[date_col])
        last_run_date = data[date_col].max() + timedelta(days=1)
        # Grouping by CustomerID
        data_process = data.groupby([id_col]).agg({
                date_col: lambda x: (last_run_date - x.max()).days,
                order_id_col: 'count',
                value_col: 'sum'})
        # Rename the columns 
        data_process.rename(columns={date_col: 'Recency',
                                 order_id_col: 'Frequency',
                                 value_col: 'MonetaryValue'}, inplace=True)


        r_labels = range(4, 0, -1); f_labels = range(1, 5)
        # Assign these labels to 4 equal percentile groups 
        r_groups = pd.qcut(data_process['Recency'].rank(method='first'), q=4, labels=r_labels, duplicates='drop')
        # Assign these labels to 4 equal percentile groups 
        f_groups = pd.qcut(data_process['Frequency'].rank(method='first'), q=4, labels=f_labels, duplicates='drop')
        # Create new columns R and F 
        data_process = data_process.assign(R = r_groups.values, F = f_groups.values)
        data_process.head()
        # Create labels for MonetaryValue
        m_labels = range(1, 5)
        # Assign these labels to four equal percentile groups 
        m_groups = pd.qcut(data_process['MonetaryValue'].rank(method='first'), q=4, labels=m_labels, duplicates='drop')
        # Create new column M
        data_process = data_process.assign(M = m_groups.values)
        return data_process


    def join_rfm(x): 
        return str(x['R']) + str(x['F']) + str(x['M'])

    #Specify the Path
    data=pd.read_csv(DatasetPath)
    data['OrderReceivedDate'] = pd.to_datetime(data['OrderReceivedDate'])
    #Change corresponding columns as required for RFM Analysis
    id_col='ShipToName'
    date_col='OrderReceivedDate'
    order_id_col='OrderNumber'
    value_col='Adj_OrderNetValue'

    data_process=process_execution_function(data,id_col,date_col, order_id_col, value_col)

    data_process['R']=data_process.loc[:,"R"].apply(lambda x : 5-x)
    data_process['F']=data_process.loc[:,"F"].apply(lambda x : 5-x)
    data_process['M']=data_process.loc[:,"M"].apply(lambda x : 5-x)
    data_process['RFM'] = data_process.apply(join_rfm, axis=1)

    data_process['RFM'] = data_process['RFM'].astype(str)
    data_process['R'] = data_process['R'].astype(str)
    data_process['F'] = data_process['F'].astype(str)
    data_process['M'] = data_process['M'].astype(str)

    data_process.loc[:,"CustomerName"]=data_process.index
    data_process=data_process.reset_index(drop=True)

    data_process.loc[:,"Segment"]='Others'
    indices=data_process[data_process.loc[:,"R"]=='1'].index
    data_process.loc[indices, "Segment"]="Recent Customers"

    indices=data_process[data_process.loc[:,"F"]=='1'].index
    data_process.loc[indices, "Segment"]="Loyal Customers"

    indices=data_process[data_process.loc[:,"M"]=='1'].index
    data_process.loc[indices,"Segment"]="Big Spenders"

    indices=data_process[data_process.loc[:,"RFM"]=='111'].index
    data_process.loc[indices,"Segment"]="Best Customers"

    indices=data_process[data_process.loc[:,"RFM"]=='311'].index
    data_process.loc[indices,"Segment"]="Almost Lost Customers"

    indices=data_process[data_process.loc[:,"RFM"]=='411'].index
    data_process.loc[indices,"Segment"]="Lost Customers"

    indices=data_process[data_process.loc[:,"RFM"]=='444'].index
    data_process.loc[indices,"Segment"]="Lost Cheap Customers"
    #data_process.to_csv("s3://sagemaker-us-east-1-754307369999/Output/RFM Data.csv", index=None)
    StringData = StringIO() 
    data_process.to_csv(StringData)
    s3_resource = boto3.resource('s3')
    s3_resource.Object(bucket_name='sagemaker-us-east-1-754307369999',key='Output/RfmOutput.csv').put(Body=StringData.getvalue())



elif model=='Churn':
    !aws s3 cp s3://sagemaker-us-east-1-754307369999/Models/Churn.py ./DataFolder/Churn.py
    !aws s3 cp s3://sagemaker-us-east-1-754307369999/ExtraFiles/requirements.txt ./DataFolder/
    from sagemaker.tensorflow import TensorFlow
    script_path='Churn.py'
    model_name='Churn'
    tf_estimator = TensorFlow(entry_point=script_path,
                              source_dir='./DataFolder',
                              base_job_name=model_name+f"-parameterName-hidden-channels-{int(time.time())}",
                              train_instance_type="ml.m5.large", 
                              role=role,
                              train_instance_count=1, 
                              framework_version='1.12', 
                              py_version='py3',
                              script_mode=True,
                              #hyperparameters={'epochs': 1}
                              hyperparameters={'filename': filename}
                               )
    tf_estimator.fit(inputs={'train': DatasetPath })
    tf_estimator.deploy(endpoint_name=model+'-endpoint-ML',instance_type='ml.m5.large',initial_instance_count=1)
    

You are using pip version 10.0.1, however version 20.1b1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 10.0.1, however version 20.1b1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
s3://ptg-banking-poc/sa_published_zone/DS1/file_type=training
part-00000-ca60b79c-6274-45e7-b75a-5a9a617f82d0.c000.csv
Arema
download: s3://sagemaker-us-east-1-754307369999/Models/Arema.py to DataFolder/Arema.py


In [3]:
!aws s3 cp s3://sagemaker-us-east-1-754307369999/ExtraFiles/requirements.txt ./DataFolder/

download: s3://sagemaker-us-east-1-754307369999/ExtraFiles/requirements.txt to DataFolder/requirements.txt


In [4]:
#!aws s3 cp s3://sagemaker-us-east-1-754307369999/Input/1500RowPublishedFile.csv /opt/ml/input/data/train/1500RowPublishedFile.csv

In [19]:
sklearn_estimator = SKLearn(entry_point=model+'.py',
                                    train_instance_type='ml.m5.large',
                                    framework_version='0.20.0',
                                    source_dir='./DataFolder',
                                    role=role,
                                    hyperparameters={'filename': filename },
                                    enable_sagemaker_metrics=True)
sklearn_estimator.fit({'train':DatasetPath})

                # Deploy my estimator to a SageMaker Endpoint and get a Predictor
predictor = sklearn_estimator.deploy(endpoint_name=model+'-endpoint--Banking-SA',instance_type='ml.m5.large',initial_instance_count=1)
    
    

2020-04-24 10:44:02 Starting - Starting the training job...
2020-04-24 10:44:04 Starting - Launching requested ML instances......
2020-04-24 10:45:10 Starting - Preparing the instances for training...
2020-04-24 10:45:54 Downloading - Downloading input data......
2020-04-24 10:46:49 Training - Training image download completed. Training in progress.2020-04-24 10:46:49,546 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2020-04-24 10:46:49,549 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-04-24 10:46:49,561 sagemaker_sklearn_container.training INFO     Invoking user training script.
2020-04-24 10:46:49,898 sagemaker-containers INFO     Module Arema does not provide a setup.py. 
Generating setup.py
2020-04-24 10:46:49,898 sagemaker-containers INFO     Generating setup.cfg
2020-04-24 10:46:49,898 sagemaker-containers INFO     Generating MANIFEST.in
2020-04-24 10:46:49,899 sagemaker-containers INFO     Installing mod

  Created wheel for wrapt: filename=wrapt-1.12.1-cp37-cp37m-linux_x86_64.whl size=66353 sha256=5d15dada0be0256e29475785cd18fb59365e55e5b07ca1a41c453e2908cd0bc1
  Stored in directory: /root/.cache/pip/wheels/b1/c2/ed/d62208260edbd3fa7156545c00ef966f45f2063d0a84f8208a
  Created wheel for absl-py: filename=absl_py-0.9.0-cp37-none-any.whl size=121932 sha256=5495c6fb9653aab66ada75ca3ec1d162d8d3f958076c46274b2e5a6f09442e37
  Stored in directory: /root/.cache/pip/wheels/8e/28/49/fad4e7f0b9a1227708cbbee4487ac8558a7334849cb81c813d
  Created wheel for pyyaml: filename=PyYAML-5.3.1-cp37-cp37m-linux_x86_64.whl size=44620 sha256=752d46f531198645c486ad5c79f14f600303cf551ef3fafd1399aba6d8b1bf3e
  Stored in directory: /root/.cache/pip/wheels/a7/c1/ea/cf5bd31012e735dc1dfea3131a2d5eae7978b251083d6247bd
Successfully built CurrencyConverter sklearn pickle-mixin Arema termcolor gast wrapt absl-py pyyaml
  Found existing installation: scipy 1.3.1
    Uninstalling scipy-1.3.1:
      Successfully uninstalled 

Fit ARIMA: (1, 1, 1)x(0, 1, 1, 12) (constant=True); AIC=1678.709, BIC=1691.784, Time=1.864 seconds
Fit ARIMA: (0, 1, 0)x(0, 1, 0, 12) (constant=True); AIC=1787.181, BIC=1792.412, Time=0.022 seconds
Fit ARIMA: (1, 1, 0)x(1, 1, 0, 12) (constant=True); AIC=1715.754, BIC=1726.214, Time=0.660 seconds
Fit ARIMA: (0, 1, 1)x(0, 1, 1, 12) (constant=True); AIC=1676.821, BIC=1687.281, Time=1.626 seconds
Near non-invertible roots for order (0, 1, 1)(0, 1, 1, 12); setting score to inf (at least one inverse root too close to the border of the unit circle: 0.990)
Fit ARIMA: (0, 1, 0)x(0, 1, 0, 12) (constant=False); AIC=1785.193, BIC=1787.808, Time=0.029 seconds
Fit ARIMA: (1, 1, 1)x(0, 1, 0, 12) (constant=True); AIC=1725.433, BIC=1735.893, Time=0.492 seconds
Fit ARIMA: (1, 1, 1)x(1, 1, 1, 12) (constant=True); AIC=1679.980, BIC=1695.671, Time=2.290 seconds
Fit ARIMA: (1, 1, 1)x(0, 1, 2, 12) (constant=True); AIC=1684.309, BIC=1699.999, Time=4.639 seconds
Near non-invertible roots for order (1, 1, 1)(0,

Training seconds: 161
Billable seconds: 161
-------------!

In [11]:
import psycopg2 as pg
import pandas.io.sql as psql
import pandas as pd
connection = pg.connect(user = 'postgres',
password = 'sa2dbroot',
host = 'sa2-db.cvl1on5n4pwi.us-east-1.rds.amazonaws.com',
port = '5432',
database = 'yash_sa_schema')
#dataframe = psql.DataFrame("SELECT * FROM category", connection)
data = pd.read_sql_query('select * from yash_sa_schema.ds1_published_table',con=connection)
print(data)
data.to_csv('./DataFolder/Trainfile.csv')
from io import StringIO # python3; python2: BytesIO 
import boto3

bucket = "sagemaker-us-east-1-754307369999" # already created on S3
csv_buffer = StringIO()
data.to_csv(csv_buffer)
s3_resource = boto3.resource('s3')
s3_resource.Object(bucket, key='Accuracy-DecisionTree/TrainFile.csv').put(Body=csv_buffer.getvalue())

          UniqueId  MarketingGroup  OrderNumber OrderType OrderReceivedDate  \
0   11745400294338              11       294338         R        02/29/2016   
1   11745400295280              11       295280         R        03/14/2016   
2   19585000734026              19       734026         R        12/30/2015   
3   19585000734027              19       734027         R        12/30/2015   
4    5234750239612              52       239612         R        01/27/2016   
5    5234750239929              52       239929         R        02/04/2016   
6    5234750240572              52       240572         R        02/25/2016   
7    5234750240811              52       240811         R        02/26/2016   
8    5234750240873              52       240873         R        02/26/2016   
9    5275540241241              52       241241         R        03/06/2016   
10  74125567689998              74       689998         R        12/23/2015   
11  74131111412968              74       412968     

{'ResponseMetadata': {'RequestId': '3C17F26AF2F96E21',
  'HostId': 'HZ0Ee/dloOwemEfHFuQFboLg51oTZ+9K0HSlelOsjvzFfh2A6S+BF6RS3JbY6h2Rw90i+n5/a94=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'HZ0Ee/dloOwemEfHFuQFboLg51oTZ+9K0HSlelOsjvzFfh2A6S+BF6RS3JbY6h2Rw90i+n5/a94=',
   'x-amz-request-id': '3C17F26AF2F96E21',
   'date': 'Fri, 24 Apr 2020 10:19:20 GMT',
   'etag': '"4c9ae2833b08e7f06b80d747be858315"',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'ETag': '"4c9ae2833b08e7f06b80d747be858315"'}

In [14]:
import psycopg2 as pg
import pandas.io.sql as psql
import pandas as pd
connection = pg.connect(user = 'postgres',
password = 'sa2dbroot',
host = 'sa2-db.cvl1on5n4pwi.us-east-1.rds.amazonaws.com',
port = '5432',
database = 'yash_sa_schema')
#dataframe = psql.DataFrame("SELECT * FROM category", connection)
data = pd.read_sql_query('select * from yash_sa_schema.sa_file_ingst_log',con=connection)
print(data)

   file_upld_log_id           file_nm file_src_cd file_type  \
0                 1  businessData.csv         DS1       csv   
1                 2     salesData.csv         DS2       csv   
2                 3      loanData.csv         DS1       csv   

                                        file_s3_path file_upld_typ  \
0  s3://ptg-banking-poc/sa_raw_zone/DS1/file_type...      Training   
1  s3://ptg-banking-poc/sa_raw_zone/DS2/file_type...       Testing   
2  s3://ptg-banking-poc/sa_raw_zone/DS1/file_type...      Training   

      file_nm_alias         file_cmnt file_upld_log_stat_cd  \
0  BusinessData.csv  Business Dataset               Started   
1     salesData.csv     Sales Dataset           Not Started   
2      LoanData.csv      Loan Dataset               Started   

                     upld_ts  upld_by raw_to_transform_flag  \
0 2020-04-08 17:16:59.926191  Default                  None   
1 2020-04-08 17:20:22.547449  Default                  None   
2 2020-04-08 17:25:33.61